In [8]:
import pandas as pd
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Graces_Project")
import re

import folium
from folium.plugins import MarkerCluster

In [22]:
df = pd.read_csv("test.csv", encoding="utf7")

In [25]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df['location'] = df['address'].apply(geocode)

df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [11]:
(df.point[0][0],df.point[0][1])

(55.8599762, -4.252606)

In [23]:
df

,address,category,link
0,Royal Exchange Square,Recording,https://www.soundcloud.com/field-recording/201...
1,167 Renfrew St,Picture,https://www.realla-images.freetls.fastly.net/r...


In [13]:
## Need to test what happens when two identical points exist.

In [14]:
def makeHref(url,link_text = None):
    if link_text == None:
        link_text = str(url)
    return '<a href="' + url + '"target="_blank">' + re.sub(r"[']+", "\\\\'", link_text[:45]) +'</a>'

In [26]:
# central coordinates of Glasgow
GLA_COORDINATES = (55.8599762, -4.252606)
  
# create empty map zoomed in on Glasgow
map = folium.Map(location=GLA_COORDINATES, zoom_start=12)

# add one markercluster per type to allow for individual toggling
recording = MarkerCluster(name='recording').add_to(map)
picture = MarkerCluster(name='picture').add_to(map)
video = MarkerCluster(name='video').add_to(map)

# add recordings to the map
for records in df[df.category == "Recording"].iterrows():
    folium.Marker(location=(records[1].point[0],records[1].point[1]), 
                  popup="<h3>Recording</h3><br>" + makeHref(records[1]['link']),
                 icon=folium.Icon(color='green', icon_color='white', icon='soundcloud', angle=0, prefix='fa'))\
    .add_to(recording)
    
# add pictures to the map
for pics in df[df.category == "Picture"].iterrows():
    folium.Marker(location=(pics[1].point[0],pics[1].point[1]), 
                  popup="<h3>Picture</h3><br>" + makeHref(pics[1]['link']),
                 icon=folium.Icon(color='blue', icon='camera', prefix='fa'))\
    .add_to(picture)
    
# # add other to the map
# for chairs in df2[df2.category == 3].iterrows():
#     folium.Marker(location=[float(chairs[1]['lat']), float(chairs[1]['lon'])], 
#                   popup=chairs[1]['Premises Name'],
#                  icon=folium.Icon(color='gray', icon='question', prefix='fa'))\
#     .add_to(other)
    
# enable toggling of data points
folium.LayerControl().add_to(map)    
    
display(map)

In [ ]:
# Because popups are just html boxes, if we can get embed codes for everything, then we're grand.